Funktioner til databehandling

In [1]:
import pandas as pd
import requests

# Funktion til at undersøge hvilke stats der er tilgængelige
def available_stats(category: int):
    X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"
    payload = {
        "operationName": "StatDetails",
        "variables": {
            "tourCode": "R",
            "statId": 0,
            "year": 2023,
            "eventQuery": None
        },
     "query": "query StatDetails($tourCode: TourCode!, $statId: String!, $year: Int, $eventQuery: StatDetailEventQuery) {\n  statDetails(\n    tourCode: $tourCode\n    statId: $statId\n    year: $year\n    eventQuery: $eventQuery\n  ) {\n    tourCode\n    year\n    displaySeason\n    statId\n    statType\n    tournamentPills {\n      tournamentId\n      displayName\n    }\n    yearPills {\n      year\n      displaySeason\n    }\n    statTitle\n    statDescription\n    tourAvg\n    lastProcessed\n    statHeaders\n    statCategories {\n      category\n      displayName\n      subCategories {\n        displayName\n        stats {\n          statId\n          statTitle\n        }\n      }\n    }\n    rows {\n      ... on StatDetailsPlayer {\n        __typename\n        playerId\n        playerName\n        country\n        countryFlag\n        rank\n        rankDiff\n        rankChangeTendency\n        stats {\n          statName\n          statValue\n          color\n        }\n      }\n      ... on StatDetailTourAvg {\n        __typename\n        displayName\n        value\n      }\n    }\n  }\n}"  
  }
    req = requests.post("https://orchestrator.pgatour.com/graphql", json=payload, headers={"x-api-key": X_API_KEY ,'name':'Simon Knobelauch Hansen', 'email':'Rfv228@alumni.ku.dk'})
    stats_in_category = req.json()["data"]["statDetails"]["statCategories"][category]
    return stats_in_category

# Funktion til at samle al ønsket data
def get_data(id_list: list):
    
    X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"
    YEAR = 2023
    first_stat = True
    
    for stat_id in id_list:
        payload = {
            "operationName": "StatDetails",
            "variables": {
                "tourCode": "R",
                "statId": stat_id,
                "year": YEAR,
                "eventQuery": None
            },
         "query": "query StatDetails($tourCode: TourCode!, $statId: String!, $year: Int, $eventQuery: StatDetailEventQuery) {\n  statDetails(\n    tourCode: $tourCode\n    statId: $statId\n    year: $year\n    eventQuery: $eventQuery\n  ) {\n    tourCode\n    year\n    displaySeason\n    statId\n    statType\n    tournamentPills {\n      tournamentId\n      displayName\n    }\n    yearPills {\n      year\n      displaySeason\n    }\n    statTitle\n    statDescription\n    tourAvg\n    lastProcessed\n    statHeaders\n    statCategories {\n      category\n      displayName\n      subCategories {\n        displayName\n        stats {\n          statId\n          statTitle\n        }\n      }\n    }\n    rows {\n      ... on StatDetailsPlayer {\n        __typename\n        playerId\n        playerName\n        country\n        countryFlag\n        rank\n        rankDiff\n        rankChangeTendency\n        stats {\n          statName\n          statValue\n          color\n        }\n      }\n      ... on StatDetailTourAvg {\n        __typename\n        displayName\n        value\n      }\n    }\n  }\n}"  
      }
        page = requests.post("https://orchestrator.pgatour.com/graphql", json=payload, headers={"x-api-key": X_API_KEY ,'name':'Simon Knobelauch Hansen', 'email':'Rfv228@alumni.ku.dk'})
        page.raise_for_status()
        data = page.json()["data"]["statDetails"]["rows"]
        # For the first stat of the loop, we can not merge dataframes, so it is initialized here
        if first_stat:
            df = pd.DataFrame(data)
            if df.empty: # Some stats have no data, giving empty dataframe, which is skipped
                continue
            col_names = ["playerId", "playerName", "country"]
            for col in range(len(page.json()["data"]["statDetails"]["statHeaders"])):
                col_name = page.json()["data"]["statDetails"]["statTitle"]+"("+page.json()["data"]["statDetails"]["statHeaders"][col]+")"
                df[col_name] = df.stats.apply(lambda x: str(x[col]['statValue']).replace(",", "").replace("$", "") if isinstance(x, list) and len(x) > 0 else None)
                col_names.append(col_name)
            df = df[col_names]
            first_stat = False
        # Here all the subsequent stats are merged onto the dataframe
        else:
            df_temp = pd.DataFrame(data)
            if df_temp.empty: # Some stats have no data, giving empty dataframe, which is skipped
                continue
            col_names = ["playerId", "playerName", "country"]
            for col in range(len(page.json()["data"]["statDetails"]["statHeaders"])):
                col_name = page.json()["data"]["statDetails"]["statTitle"]+"("+page.json()["data"]["statDetails"]["statHeaders"][col]+")"
                df_temp[col_name] = df_temp.stats.apply(lambda x: str(x[col]['statValue']).replace(",", "").replace("$", "") if isinstance(x, list) and len(x) > 0 else None)
                col_names.append(col_name)
            df_temp = df_temp[col_names]

            df = pd.merge(df, df_temp, left_on=["playerId", "playerName", "country"], right_on=["playerId", "playerName", "country"], how="outer")
            
    return df

Tjek hvilke stats (statId) fra hver af de 9 kategories der skal med. Inputtet til funktionen kan være mellem 0 og 8.

In [2]:
available_stats(category=0)

{'category': 'STROKES_GAINED',
 'displayName': 'Strokes Gained',
 'subCategories': [{'displayName': 'Strokes Gained',
   'stats': [{'statId': '02675', 'statTitle': 'SG: Total'},
    {'statId': '02674', 'statTitle': 'SG: Tee-to-Green'},
    {'statId': '02567', 'statTitle': 'SG: Off-the-Tee'},
    {'statId': '02568', 'statTitle': 'SG: Approach the Green'},
    {'statId': '02569', 'statTitle': 'SG: Around-the-Green'},
    {'statId': '02564', 'statTitle': 'SG: Putting'}]}]}

Definér liste med Id'er på de stats der skal med, og kør funtionen for at få en dataframe. Eksempel på liste vist.
Længden af dataframen skal være lig med antallet af unikke playerIds, for at sikre sig at hver række svare til én spiller.

In [3]:
desired_stats = ["159", "317", "02402", "190", "02338", "02339", "02340", "02430", "362", "104", "115", "120", "299", "107", "106", "105", "137", "138", "014", "154", "194", "02337", "02447", "02394", "127", "186", "085", "086", "087", "088", "02351", "02356"]
df = get_data(id_list=desired_stats)
print("Length of df (%s) should be the amount of unique player ids which is: %s" % (len(df), len(df.playerId.unique())))
df

Length of df (1671) should be the amount of unique player ids which is: 1671


,playerId,playerName,country,Longest Drives(Distance),Longest Drives(Tournament),Longest Drives(Round),Longest Drives(Hole),Longest Drives(Course),Driving Distance - All Drives(All Drives),Driving Distance - All Drives(Total Distance),...,Percentage of potential money won(Potential Money),FedexCup Regular Season Points(Points),FedexCup Regular Season Points(# of Wins),FedexCup Regular Season Points(# of Top 10's),FedexCup Regular Season Points(Points behind Lead),All-Around Ranking(Total),Official World Golf Ranking(Avg Points),Official World Golf Ranking(Total Points),Official World Golf Ranking(Points lost),Official World Golf Ranking(Points gained)
0,27129,Luke List,United States,459,Sentry,3,7,656,304.8,287146,...,60831700,309,0,0,3011,880,1.01,52.51,-40.55,23.11
1,28420,Ryan Brehm,United States,455,Sentry,3,18,656,299.9,243547,...,53838000,118,0,0,3202,1111,.30,15.59,-10.01,7.46
2,46046,Scottie Scheffler,United States,451,Sentry,4,18,656,302.2,259895,...,56459000,3146,2,15,174,297,11.96,597.97,-236.99,397.78
3,39324,J.J. Spaun,United States,446,Sentry,4,18,656,293.2,263856,...,59804700,619,0,3,2701,724,1.37,71.35,-38.84,34.48
4,49964,Aaron Wise,United States,439,Sentry,3,7,656,295.3,152945,...,37250000,289,0,1,3031,726,1.50,65.91,-54.11,6.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,60049,Adam Breen,South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,.09,3.71,-1.39,1.46
1667,47970,Sungmin Cho,South Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,.09,3.78,-2.05,.81
1668,22078,Mikael Lundberg,Sweden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,.09,3.96,-2.20,1.59
1669,26418,Adam Bland,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,.09,3.68,-1.21,1.15


Optional: Hvis dataframen skal gemmes som en .csv fil, så den hurtigt kan loades senere uden at genkøre ovenstående

In [9]:
# Write .csv
df.to_csv("golf_data.csv", index=False)
# Read .csv
df = pd.read_csv("golf_data.csv")

,playerId,playerName,country,Longest Drives(Distance),Longest Drives(Tournament),Longest Drives(Round),Longest Drives(Hole),Longest Drives(Course),Driving Distance - All Drives(All Drives),Driving Distance - All Drives(Total Distance),...,Percentage of potential money won(Potential Money),FedexCup Regular Season Points(Points),FedexCup Regular Season Points(# of Wins),FedexCup Regular Season Points(# of Top 10's),FedexCup Regular Season Points(Points behind Lead),All-Around Ranking(Total),Official World Golf Ranking(Avg Points),Official World Golf Ranking(Total Points),Official World Golf Ranking(Points lost),Official World Golf Ranking(Points gained)
0,27129.0,Luke List,United States,459.0,Sentry,3.0,7.0,656.0,304.8,287146.0,...,60831700.0,309.0,0.0,0.0,3011.0,880.0,1.01,52.51,-40.55,23.11
1,28420.0,Ryan Brehm,United States,455.0,Sentry,3.0,18.0,656.0,299.9,243547.0,...,53838000.0,118.0,0.0,0.0,3202.0,1111.0,0.30,15.59,-10.01,7.46
2,46046.0,Scottie Scheffler,United States,451.0,Sentry,4.0,18.0,656.0,302.2,259895.0,...,56459000.0,3146.0,2.0,15.0,174.0,297.0,11.96,597.97,-236.99,397.78
3,39324.0,J.J. Spaun,United States,446.0,Sentry,4.0,18.0,656.0,293.2,263856.0,...,59804700.0,619.0,0.0,3.0,2701.0,724.0,1.37,71.35,-38.84,34.48
4,49964.0,Aaron Wise,United States,439.0,Sentry,3.0,7.0,656.0,295.3,152945.0,...,37250000.0,289.0,0.0,1.0,3031.0,726.0,1.50,65.91,-54.11,6.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,60049.0,Adam Breen,South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.09,3.71,-1.39,1.46
1667,47970.0,Sungmin Cho,South Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.09,3.78,-2.05,0.81
1668,22078.0,Mikael Lundberg,Sweden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.09,3.96,-2.20,1.59
1669,26418.0,Adam Bland,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.09,3.68,-1.21,1.15
